In [32]:
import os
%load_ext autoreload
%autoreload 2
os.chdir('/home/users/stariq/Codes/cudaq-qrc')
!jupyter nbextension enable --py widgetsnbextension

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [33]:
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.ensemble import RandomForestRegressor, GradientBoostingClassifier
from sklearn.metrics import mean_squared_error
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
import os
from sklearn.preprocessing import MinMaxScaler
from dynamics import *
import re
import math
from optread import *
from scipy.stats import binom
#set latex font for plots
plt.rc('text', usetex=True)
plt.rc('font', family='serif')

In [34]:
# ========= utilities for fair-size baselines =================================
def set_seed(seed):
    import random, numpy as np, torch
    random.seed(seed); np.random.seed(seed)
    torch.manual_seed(seed); torch.cuda.manual_seed_all(seed)

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

def pick_emb_dim(budget, n_feat, nhead=4, nlayers=2, ff_mult=4, tol=.10):
    """
    Return the largest emb_dim (multiple of nhead) whose 2-layer Transformer
    fits within `budget·(1+tol)` params.
    """
    def n_params(d):
        mha = 3*d*d*nlayers            # Q,K,V
        out = d*d   *nlayers           # W_O
        ff  = 2*d*ff_mult*d*nlayers    # W1,W2
        ln  = 2*d*nlayers              # layer norms
        bias= d*(ff_mult*2*nlayers + nlayers + 1)  # FF + attn + regressor
        pos = d*(n_feat+1)             # pos + CLS
        return mha+out+ff+ln+bias+pos
    d = max(nhead, nhead*((budget//n_feat)//nhead))
    while n_params(d) > budget*(1+tol):
        d -= nhead
        if d < nhead: raise ValueError("budget too small")
    return d


In [35]:
N_classes = 4
N_features = 4
N_samples  = 500

# 1) Generate a multiclass classification dataset
X, y = make_classification(
    n_samples    = N_samples,
    n_features   = N_features,
    n_repeated   = 0,
    n_classes    = N_classes,
    n_clusters_per_class = 1,
    class_sep    = 1.0,
    flip_y       = 0.01,
    random_state = 402
)

# 2) Normalize features to [0, 1]
scaler_X      = MinMaxScaler(feature_range=(0, 1))
X_normalized = scaler_X.fit_transform(X)

# 3) Split: 60% train, 40% temp (test+val)
X_train, X_temp, y_train, y_temp = train_test_split(
    X_normalized, y,
    test_size   = 0.4,
    shuffle     = True,
    random_state= 402
)

# 4) Split the 40% temp equally into validation (20%) and test (20%)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp,
    test_size   = 0.5,
    shuffle     = True,
    random_state= 402
)

print("Shapes:",
      "X_train", X_train.shape,
      "X_val",   X_val.shape,
      "X_test",  X_test.shape,
      "y_train", y_train.shape,
      "y_val",   y_val.shape,
      "y_test",  y_test.shape)

Shapes: X_train (300, 4) X_val (100, 4) X_test (100, 4) y_train (300,) y_val (100,) y_test (100,)


In [36]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import os
import torch.nn.functional as F

#from dynamics import embeddings_multi_gpu_auto as emb_mgpu

# Convert to torch tensors
X_train_t = torch.tensor(X_train, dtype=torch.float32)
X_test_t  = torch.tensor(X_test,  dtype=torch.float32)
y_train_t = torch.tensor(y_train, dtype=torch.long)
y_test_t  = torch.tensor(y_test,  dtype=torch.long)


In [37]:
#gpu count
gpu_count = torch.cuda.device_count()
print("Number of GPUs available:", gpu_count)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Number of GPUs available: 2
Using device: cuda


In [38]:
# --------------------------------------------------------------------- #
# Random-Fourier baseline in “dim-tunable” form                        #
# --------------------------------------------------------------------- #
class RandomFourierClassifier(nn.Module):
    """
    φ(x) = √(2/D) cos(x·W + b)  →  linear → logits over n_classes
    """
    def __init__(self, in_dim: int, out_dim: int, n_classes: int):
        super().__init__()
        self.out_dim   = out_dim
        self.register_buffer("W", torch.randn(in_dim, out_dim) * 0.1)
        self.register_buffer("b", torch.randn(out_dim) * 2 * math.pi)
        # now outputs n_classes logits
        self.linear = nn.Linear(out_dim, n_classes)

    def forward(self, x):  # x : (B, F)
        phi    = torch.cos(x @ self.W + self.b) * math.sqrt(2 / self.out_dim)
        logits = self.linear(phi)               # (B, n_classes)
        return logits


# --------------------------------------------------------------------- #
# Constructor that _search_dim_multiple_ can call                       #
# --------------------------------------------------------------------- #
def make_rff(dim):
    return RandomFourierClassifier(
        in_dim    = X_train_t.shape[1],
        out_dim   = dim,
        n_classes = N_classes
    ).to(device)

    
# --------------------------------------------------------------------- #
# Trainable-parameter count helper (buffers are ignored automatically)  #
# --------------------------------------------------------------------- #
def n_params(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [39]:
class MGTransformerClassifier(nn.Module):
    def __init__(self,
                 n_features: int,
                 emb_dim: int = 64,
                 nhead: int = 4,
                 nhid: int = 512,
                 nlayers: int = 4,
                 dropout: float = 0.1,
                 n_classes: int = 4):
        super().__init__()
        self.feat_proj = nn.Linear(1, emb_dim)
        self.cls_token = nn.Parameter(torch.randn(1, 1, emb_dim))
        self.pos_emb   = nn.Parameter(torch.randn(1, n_features+1, emb_dim))

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=emb_dim, nhead=nhead,
            dim_feedforward=nhid,
            dropout=dropout,
            batch_first=True
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, nlayers)
        self.classifier  = nn.Linear(emb_dim, n_classes)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        B, F = x.shape
        h = self.feat_proj(x.unsqueeze(-1))      # (B, F, E)
        cls = self.cls_token.expand(B, -1, -1)   # (B, 1, E)
        h   = torch.cat([cls, h], dim=1)         # (B, F+1, E)
        h  += self.pos_emb                       # add pos emb
        h   = self.transformer(h)                # (B, F+1, E)
        cls_emb = h[:,0]                         # (B, E)
        return self.classifier(cls_emb)          # (B, n_classes)


In [ ]:
def train_classifier(model, loader_tr, *,
                     epochs       = 100,
                     lr           = 1e-3,
                     weight_decay = 1e-4,
                     patience     = 10,
                     desc         = "train",
                     early_stop   = True):
    """
    Train a classifier (supports both (x,y) and (x, reservoir, y) batches).
    Returns list of training cross-entropy losses per epoch.
    """
    opt = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    best_state, best_loss, wait = None, float("inf"), 0
    train_losses = []

    bar = tqdm(range(1, epochs+1), desc=f"{desc}: CE=∞", leave=False)
    for ep in bar:
        model.train()
        running = 0.0
        for batch in loader_tr:
            # unpack batch
            if len(batch) == 3:
                xb, reservoir, yb = batch
                reservoir = reservoir.to(device)
            else:
                xb, yb = batch
                reservoir = None
            xb, yb = xb.to(device), yb.to(device)

            # forward
            logits = model(xb, reservoir) if reservoir is not None else model(xb)
            loss   = F.cross_entropy(logits, yb)

            # step
            opt.zero_grad()
            loss.backward()
            opt.step()
            running += loss.item() * xb.size(0)

        ce = running / len(loader_tr.dataset)
        train_losses.append(ce)
        bar.set_description(f"{desc}: CE={ce:.4f}")

        # early stopping on training loss
        if ce < best_loss:
            best_loss, best_state, wait = ce, model.state_dict().copy(), 0
        else:
            wait += 1
            if early_stop and wait >= patience:
                break

    bar.close()
    model.load_state_dict(best_state)
    return train_losses


def predict_classifier(model, loader):
    """
    Inference supporting both (x,y) and (x, reservoir, y) batches.
    Returns (preds, trues) as LongTensors.
    """
    model.eval()
    preds, trues = [], []
    with torch.no_grad():
        for batch in loader:
            if len(batch) == 3:
                xb, reservoir, yb = batch
                reservoir = reservoir.to(device)
            else:
                xb, yb = batch
                reservoir = None
            xb = xb.to(device)
            logits = model(xb, reservoir) if reservoir is not None else model(xb)
            preds.append(logits.argmax(dim=1).cpu())
            trues.append(yb)
    return torch.cat(preds), torch.cat(trues)

In [41]:
# ——————— Prepare DataLoaders ———————
train_ds = TensorDataset(X_train_t, y_train_t)
test_ds  = TensorDataset(X_test_t,  y_test_t)
loader_tr = DataLoader(train_ds, batch_size=32, shuffle=True)
loader_te = DataLoader(test_ds,  batch_size=32, shuffle=False)

In [42]:
#print alll available devices
print("Available devices:")
for i in range(torch.cuda.device_count()):
    print(f"Device {i}: {torch.cuda.get_device_name(i)}")

Available devices:
Device 0: Tesla V100-SXM2-16GB
Device 1: Tesla V100-SXM2-16GB


In [43]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"  # Use GPUs 0, 1, 2, and 3

In [44]:
all_train_emb, all_test_emb = {}, {}

for n in range(4, 5):          # 2 … 10 qubits
    tr_emb, te_emb = get_embeddings_for_nsites(
        nsites      = n,
        X_train     = X_train,     # your current arrays
        X_test      = X_test,
        dataset_tag = f"{N_classes}-{N_features}feat-classify"   # or "MackeyGlass", etc.
    )
    all_train_emb[n] = tr_emb
    all_test_emb[n]  = te_emb


✓ n=4: loaded cached embeddings


In [45]:
#testing for n = 2
nsites = n
readouts = generate_readouts(nsites)
train_embeddings = all_train_emb[nsites]
test_embeddings = all_test_emb[nsites]
T_res = 6
R = len(readouts)
print("train_flat.shape:", train_embeddings.shape)
print("test_flat.shape: ", test_embeddings.shape)

train_flat.shape: (300, 180)
test_flat.shape:  (100, 180)


In [46]:
# # 3) reshape into 3D
# N, D = train_embeddings.shape
# N_test, D_test = test_embeddings.shape
# train3 = train_embeddings.reshape(N, T_res, R)
# test3  = test_embeddings.reshape(N_test, T_res, R)


# # inject a realistic cocktail of errors
# train_noisy3 = noisy_embeddings(train3, gaussian_sigma=0.2,
#                                 multiplicative_sigma=0.01,
#                                 shots=800,
#                                 T2=3.0, dt=0.5)
# test_noisy3  = noisy_embeddings(test3,  gaussian_sigma=0.2,
#                                 multiplicative_sigma=0.01,
#                                 shots=800,
#                                 T2=3.0, dt=0.5)


In [47]:
torch.manual_seed(0)
# reshape to (batch, T, R)
R = len(readouts)
T = train_embeddings.shape[1] // R
res_tr = train_embeddings.reshape(-1, T, R)
res_te = test_embeddings.reshape(-1, T, R)

# res_tr =selected_train_embeddings.reshape(-1, T, R)
# res_te =selected_test_embeddings.reshape(-1, T, R)
# to tensors
res_tr_t = torch.tensor(res_tr, dtype=torch.float32)
res_te_t = torch.tensor(res_te, dtype=torch.float32)
ld_qr_tr = DataLoader(
    TensorDataset(X_train_t, res_tr_t, y_train_t),
    batch_size=32, shuffle=True
)
ld_qr_te = DataLoader(
    TensorDataset(X_test_t,  res_te_t,  y_test_t),
    batch_size=32
)


In [48]:
# #count parameters
# print("Reservoir-Attention Model params:", count_parameters(model2))

In [49]:
class QRCrossAttnLayer(nn.Module):
    """
    Cross-attention from the reservoir onto *all* tokens.
    reservoir : (B,T,R)
    tokens    : (B,F+1,E)
    x_feat_emb: (B,E)  – mean-pooled scalar embedding
    """
    def __init__(self, emb_dim, res_dim, ff_mult=1, dropout=0.0):
        super().__init__()
        self.score_proj = nn.Linear(res_dim, 1)      # (T,R) → (T,1)
        self.ff = nn.Sequential(
            nn.Linear(emb_dim, ff_mult * emb_dim),
            nn.GELU(),
            nn.Linear(ff_mult * emb_dim, emb_dim),
        )
        self.norm1 = nn.LayerNorm(emb_dim)
        self.norm2 = nn.LayerNorm(emb_dim)

    def forward(self, tokens, reservoir, x_feat_emb):
        # 1) attention weights from reservoir
        attn    = torch.softmax(self.score_proj(reservoir), dim=1)   # (B,T,1)

        # 2) value vector = feature embedding (no extra proj)
        context = (attn * x_feat_emb.unsqueeze(1)).sum(1)            # (B,E)

        # 3) add to *every* token, then FFN
        tokens  = self.norm1(tokens + context.unsqueeze(1))
        tokens  = self.norm2(tokens + self.ff(tokens))
        return tokens




class MGTransformerQRClassifier(nn.Module):
    def __init__(self,
                 n_features: int,
                 res_dim:     int,
                 emb_dim:     int,
                 n_classes:   int = 4,
                 ff_mult:     int = 4,
                 dropout:     float = 0.0):
        super().__init__()
        self.feat_proj = nn.Linear(n_features, emb_dim)
        self.layer     = QRCrossAttnLayer(emb_dim, res_dim,
                                          ff_mult=ff_mult,
                                          dropout=dropout)
        self.classifier= nn.Linear(emb_dim, n_classes)

    def forward(self, x, reservoir):
        feat_emb = self.feat_proj(x)              # (B, E)
        tokens   = feat_emb.unsqueeze(1)          # (B, 1, E)
        tokens   = self.layer(tokens, reservoir, feat_emb)
        cls_emb  = tokens.squeeze(1)              # (B, E)
        return self.classifier(cls_emb)           # (B, n_classes)








In [50]:
# ════════════════════════════════════════════════════════════════════
# 1️⃣  parameter counter & emb-dim search
# ════════════════════════════════════════════════════════════════════
def n_params(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

def search_dim_multiple(
        ctor, *, target,  nhead=4,  low=8, high=512, tol=0.05, **kw):
    """
    Binary-search emb_dim (multiple of nhead) so that
    |#params - target| / target  ≤ tol
    """
    low  = math.ceil(low  / nhead) * nhead
    high = math.floor(high / nhead) * nhead
    best, best_p = None, float("inf")

    while low <= high:
        mid   = (low + high) // (2*nhead) * nhead  # rounded to multiple
        model = ctor(mid, **kw);  p = n_params(model)

        if abs(p-target) <= tol*target:   # hit!
            return model, p
        if p < target: low  = mid + nhead
        else:          high = mid - nhead

        if abs(p-target) < abs(best_p-target):
            best, best_p = model, p
    return best, best_p                        # closest found


# ════════════════════════════════════════════════════════════════════
# 2️⃣  constructors with **identical MLP width 4×E** and depth = 1
# ════════════════════════════════════════════════════════════════════
def make_transformer(E):
    return MGTransformerClassifier(
        n_features = X_train_t.shape[1],
        emb_dim    = E,
        nhead      = 4,
        nhid       = 4*E,
        nlayers    = 1,
        dropout    = 0.0,
        n_classes  = N_classes
    ).to(device)


def make_qt(E):
    return MGTransformerQRClassifier(
        n_features= X_train_t.shape[1],
        res_dim    = R,
        emb_dim    = E,
        n_classes  = N_classes,
        ff_mult    = 4,
        dropout    = 0.0
    ).to(device)


def make_rff(D):
    return RandomFourierClassifier(
        in_dim    = X_train_t.shape[1],
        out_dim   = D,
        n_classes = N_classes
    ).to(device)



# ════════════════════════════════════════════════════════════════════
# 3️⃣  pick dimensions so every model ≈ TARGET params
# ════════════════════════════════════════════════════════════════════
TARGET = 1_600        # you can change this in one place

model_tf , p_tf  = search_dim_multiple(make_transformer,
                                       target=TARGET, nhead=4)
model_rff, p_rff = search_dim_multiple(make_rff,
                                       target=TARGET, nhead=1,
                                       low=32, high=12_000)

model_qt , p_qt  = search_dim_multiple(make_qt,
                                       target=TARGET, nhead=4)
print(f"Quantum-Transformer : {p_qt :>5} params  (E={model_qt.feat_proj.out_features})")

print(f"Transformer         : {p_tf :>5} params  (E={model_tf.feat_proj.out_features})")

print(f"Random Fourier      : {p_rff:>5} params  (D={model_rff.out_dim})")

Quantum-Transformer :  1403 params  (E=12)
Transformer         :  2032 params  (E=12)
Random Fourier      :  1624 params  (D=405)


In [51]:
# 1) Train each model
loss_tf  = train_classifier(model_tf,  loader_tr, epochs=100, lr=1e-3)
loss_rff = train_classifier(model_rff, loader_tr, epochs=100, lr=1e-3)
loss_qt  = train_classifier(model_qt,  ld_qr_tr, epochs=100, lr=1e-3)

# 2) Evaluate on test set
pred_tf,  true_tf  = predict_classifier(model_tf,  loader_te)
pred_rff, true_rff = predict_classifier(model_rff, loader_te)
pred_qt,  true_qt  = predict_classifier(model_qt,  ld_qr_te)

from sklearn.metrics import accuracy_score, classification_report
print("Transformer Test Acc:", accuracy_score(true_tf,  pred_tf))
print("RFF         Test Acc:", accuracy_score(true_rff, pred_rff))
print("Quantum-Transformer Test Acc:", accuracy_score(true_qt,  pred_qt))

print("\nClassification report (Transformer):")
print(classification_report(true_tf, pred_tf))


train: CE=∞:   0%|          | 0/100 [00:00<?, ?it/s]

train: CE=∞:   0%|          | 0/100 [00:00<?, ?it/s]

train: CE=∞:   0%|          | 0/100 [00:00<?, ?it/s]

ValueError: too many values to unpack (expected 2)

Quantum-Transformer :  1466 params  (E=12)


QT: loss=∞:   0%|          | 0/300 [00:00<?, ?it/s]